### Required Libraries

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


### Load Dataset

In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
model_data.shape

(478328, 2831)

### Drop Null target records

In [4]:
model_data['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
NaN    354961
0.0    109935
1.0     13432
Name: count, dtype: int64

In [5]:
# Drop Null values of Target 
df = model_data.dropna(subset=['60_in_15mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

### Split the Time interval

In [6]:
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2023-03')]
oot_df1= data[(data['month_year']>='2023-04') & (data['month_year']<='2023-06')]

In [7]:
model_df1.shape, oot_df1.shape

((101984, 2831), (17343, 2831))

In [8]:
features_list=[
 'distinct_merchant',
 'SUM(raw.amount WHERE action = 1)/SUM(raw.amount)#ever',
 'MEAN(raw.amount WHERE device_os_category = Windows)#ever',
 'NUM_UNIQUE(pg_success_raw_data.month_end_start_category)',
 'max_one_time_spent',
 'amt_first_android_txn',
 'MAX(pg_success_raw_data.amount)',
 'top_payment_mode_count',
 'amt_first_cc_txn',
               'COUNT(raw WHERE amount_category = 6)/COUNT(raw)#ever',
               'ENTROPY(raw.mode_cleaned)#720',
               'COUNT(pg_success_raw_data WHERE revised_merchant_category = bill_pay)',
               'YEAR(first_pg_failed_raw_data_time)',
 'month_year',
 '60_in_15mob']

In [9]:
# Selected Features for model build purpose
model_df = model_df1[features_list]

In [10]:
oot_df = oot_df1[features_list]

In [11]:
model_df.shape, oot_df.shape

((101984, 15), (17343, 15))

In [12]:
model_df['month_year'].isnull().sum()

np.int64(0)

In [13]:
#Handling Null Values
for i in model_df.columns:
    model_df[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3036121/2889689820.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  model_df[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3036121/2889689820.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df[i].fillna(-9999,inplace=True)


In [14]:
#Handling Null Values
for i in oot_df.columns:
    oot_df[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3036121/1074864086.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  oot_df[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3036121/1074864086.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oot_df[i].fillna(-9999,inplace=True)


In [15]:
model_df['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
0.0    90856
1.0    11128
Name: count, dtype: int64

In [16]:
oot_df['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
0.0    15505
1.0     1838
Name: count, dtype: int64

In [17]:
#Takeout record where banking data only present(exclude -9999)
model_df = model_df[(model_df !=-9999).all(axis=1)]

In [18]:
model_df.shape

(37638, 15)

In [19]:
oot_df = oot_df[(oot_df !=-9999).all(axis=1)]

In [20]:
oot_df.shape

(6426, 15)

In [21]:
# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '60_in_15mob']) # Adjust 'target' to your actual target column name
y_model = model_df['60_in_15mob']

In [22]:
X_oot = oot_df.drop(columns=['month_year', '60_in_15mob'])
y_oot = oot_df['60_in_15mob']

In [23]:
X_oot.shape,y_oot.shape

((6426, 13), (6426,))

In [24]:
X_model.shape

(37638, 13)

### Split Model dataset into Train and Test

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [26]:
X_train.shape, X_test.shape

((22582, 13), (15056, 13))

In [27]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(18065, 13) (4517, 13) (15056, 13)


In [28]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [29]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
    'max_depth': 2,
    'subsample': 0.4,
    'colsample_bytree': 0.4,
    'lambda': 4.7,
    'alpha': 1.8,
    'random_state': 42  
}


evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 800,
    early_stopping_rounds= 50,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.52875	eval-auc:0.50653
[10]	train-auc:0.55610	eval-auc:0.54295
[20]	train-auc:0.57166	eval-auc:0.54372
[30]	train-auc:0.57435	eval-auc:0.54889
[40]	train-auc:0.57535	eval-auc:0.55490
[50]	train-auc:0.57841	eval-auc:0.55934
[60]	train-auc:0.58002	eval-auc:0.55915
[70]	train-auc:0.58004	eval-auc:0.55949
[80]	train-auc:0.58010	eval-auc:0.56140
[90]	train-auc:0.58076	eval-auc:0.56342
[100]	train-auc:0.58109	eval-auc:0.56399
[110]	train-auc:0.58131	eval-auc:0.56351
[120]	train-auc:0.58340	eval-auc:0.56551
[130]	train-auc:0.58378	eval-auc:0.56538
[140]	train-auc:0.58441	eval-auc:0.56593
[150]	train-auc:0.58535	eval-auc:0.56674
[160]	train-auc:0.58608	eval-auc:0.56809
[170]	train-auc:0.58679	eval-auc:0.56852
[180]	train-auc:0.58778	eval-auc:0.56827
[190]	train-auc:0.58833	eval-auc:0.56947
[200]	train-auc:0.58915	eval-auc:0.56869
[210]	train-auc:0.58992	eval-auc:0.56931
[220]	train-auc:0.59020	eval-auc:0.56921
[230]	train-auc:0.59009	eval-auc:0.56860
[240]	train-auc:0.59077	eva

In [30]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.60, Gini: 0.19, KS: 0.14
Eval AUC: 0.57, Gini: 0.14, KS: 0.11
Test AUC: 0.55, Gini: 0.10, KS: 0.08
OOT AUC: 0.57, Gini: 0.14, KS: 0.12


In [31]:
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.4f}, KS: {train_ks:.4f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.4f}, KS: {test_ks:.4f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.4f}, KS: {oot_ks:.4f}")

Train AUC: 0.60, Gini: 0.1905, KS: 0.1422
Test AUC: 0.55, Gini: 0.0986, KS: 0.0818
OOT AUC: 0.57, Gini: 0.1421, KS: 0.1157


In [32]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                              Feature  Importance
5                               amt_first_android_txn       124.0
1   SUM(raw.amount WHERE action = 1)/SUM(raw.amoun...       121.0
10                      ENTROPY(raw.mode_cleaned)#720       118.0
4                                  max_one_time_spent       114.0
2   MEAN(raw.amount WHERE device_os_category = Win...       113.0
12                YEAR(first_pg_failed_raw_data_time)        93.0
9   COUNT(raw WHERE amount_category = 6)/COUNT(raw...        65.0
8                                    amt_first_cc_txn        56.0
0                                   distinct_merchant        54.0
7                              top_payment_mode_count        53.0
11  COUNT(pg_success_raw_data WHERE revised_mercha...        45.0
6                     MAX(pg_success_raw_data.amount)        44.0
3   NUM_UNIQUE(pg_success_raw_data.month_end_start...        29.0


In [33]:
feature_importance_df

Feature  Importance
5                               amt_first_android_txn       124.0
1   SUM(raw.amount WHERE action = 1)/SUM(raw.amoun...       121.0
10                      ENTROPY(raw.mode_cleaned)#720       118.0
4                                  max_one_time_spent       114.0
2   MEAN(raw.amount WHERE device_os_category = Win...       113.0
12                YEAR(first_pg_failed_raw_data_time)        93.0
9   COUNT(raw WHERE amount_category = 6)/COUNT(raw...        65.0
8                                    amt_first_cc_txn        56.0
0                                   distinct_merchant        54.0
7                              top_payment_mode_count        53.0
11  COUNT(pg_success_raw_data WHERE revised_mercha...        45.0
6                     MAX(pg_success_raw_data.amount)        44.0
3   NUM_UNIQUE(pg_success_raw_data.month_end_start...        29.0

In [34]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.169895         0.129810        0.174738        0.117308   1807   
9          0.115172         0.112601        0.117296        0.108778   1806   
8          0.113448         0.105972        0.108775        0.103405   1807   
7          0.104097         0.101202        0.103402        0.099132   1806   
6          0.089701         0.097353        0.099132        0.095668   1806   
5          0.091865         0.094065        0.095666        0.092450   1807   
4          0.068660         0.090979        0.092448        0.089436   1806   
3          0.078583         0.087821        0.089433        0.086160   1807   
2          0.076966         0.084281        0.086156        0.082187   1806   
1          0.051467         0.078850        0.082186        0.071450   1807   

        label_good  label_bad  
Decile                         
10            1500        307  
9             1598        208  
8             1602        205  
7             1618        188  
6             1644        162  
5             1641        166  
4             1682        124  
3             1665        142  
2             1667        139  
1             1714         93

In [35]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.122842         0.129702        0.170811        0.117098   1506   
9          0.108970         0.112159        0.117082        0.108238   1505   
8          0.105578         0.105566        0.108233        0.103004   1506   
7          0.101661         0.100819        0.103002        0.098866   1505   
6          0.090305         0.097151        0.098864        0.095547   1506   
5          0.078405         0.093909        0.095546        0.092344   1505   
4          0.104914         0.090801        0.092343        0.089244   1506   
3          0.076412         0.087646        0.089243        0.085995   1505   
2          0.072377         0.084155        0.085992        0.082081   1506   
1          0.074369         0.078885        0.082080        0.070804   1506   

        label_good  label_bad  
Decile                         
10            1321        185  
9             1341        164  
8             1347        159  
7             1352        153  
6             1370        136  
5             1387        118  
4             1348        158  
3             1390        115  
2             1397        109  
1             1394        112

In [36]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.136858         0.134383        0.177261        0.121430    643   
9          0.133956         0.116139        0.121416        0.112257    642   
8          0.115086         0.109048        0.112234        0.106307    643   
7          0.107477         0.104081        0.106298        0.101983    642   
6          0.104199         0.100168        0.101964        0.098431    643   
5          0.090343         0.096666        0.098424        0.094925    642   
4          0.088647         0.093186        0.094920        0.091506    643   
3          0.068536         0.089704        0.091504        0.087957    642   
2          0.090202         0.085891        0.087955        0.083749    643   
1          0.065319         0.080323        0.083733        0.070950    643   

        label_good  label_bad  
Decile                         
10             555         88  
9              556         86  
8              569         74  
7              573         69  
6              576         67  
5              584         58  
4              586         57  
3              598         44  
2              585         58  
1              601         42

In [37]:
#Save the Model
xgb_model.save_model('LeoPayu_PG_only_60_in_15mob.json')
print('Model Saved')

Model Saved
